In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import pandas as pd
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import time
import re
from progressbar import ProgressBar
from dateutil.parser import parse
import datetime

In [2]:
#Set headers
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'}

# data load

In [6]:
import twint
import nest_asyncio

import twint
import nest_asyncio

d_begin = "2022-03-12"
d_end = "2022-04-02"
daily_datarange = pd.date_range(start=d_begin,
                                end=d_end ,
                                freq='b')
nest_asyncio.apply()
tweet_ids = []
tweet_urls  = []

for start_i in range(0, len(daily_datarange), 1):
    c = twint.Config()
    c.Username = "washingtonpost"   
    c.Pandas = True
    c.Since = str(daily_datarange[start_i].strftime("%Y-%m-%d"))
    c.Until = str((daily_datarange[start_i] + datetime.timedelta(days=2)).strftime("%Y-%m-%d"))
    c.Store_csv = True
    c.Custom_csv = ["id", "user_id", "username", "tweet"]
    c.Output = ("C:/Users/13a71/Dropbox/github/crawler/tweeter crawler/twint/wp/{}.csv".format(c.Since))
    c.Filter_retweets = True
    # Run
    twint.run.Search(c)
    
    
    tweet_urls.extend([str(tweet.urls) for tweet in tweet_ids])


[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
1504603897993834497 2022-03-18 08:41:38 +0900 <washingtonpost> Costly and years late, NASA’s SLS moon rocket rolls to the launchpad for the first time  https://t.co/kcaSCYzGg8
1504599137756631046 2022-03-18 08:22:43 +0900 <washingtonpost> Texas man, temporarily lost in system, pleads guilty to assaulting police on Jan. 6  https://t.co/AimkEzgNPt
1504594788259090434 2022-03-18 08:05:26 +0900 <washingtonpost> Dreading the knock at the door: Parents of trans kids in Texas are terrified for their families  https://t.co/Mkw3xwvdwP
1504590138562732036 2022-03-18 07:46:57 +0900 <washingtonpost> Ryan Zimmerman, settling into retirement, is discovering what’s next  https://t.co/lazIFlQ4j0
1504585257839910914 2022-03-18 07:27:33 +0900 <washingtonpost> More than two dozen Senate Republicans demand Biden do more for Ukraine after voting a

In [10]:
wp1 = pd.read_excel('~/documents/wp_raw_tweet0402.xlsx').urls

In [11]:
#Open/reset progressbar and dataframes
pbar = ProgressBar()

#Create dataframe for storing article contents
content_df = pd.DataFrame(columns=['date', 'title', 'subtitle', 'category', 'author', 'content', 'url'])

#Create dataframe for errorlist
error_df = pd.DataFrame(columns=['url'])

# module

In [12]:

def to_string(instance):
    final_string = ''
    if isinstance(instance, NavigableString):
        return instance
    for contents in instance.contents:
        final_string += to_string(contents)
    return final_string

import re
import requests
from datetime import datetime
from bs4 import BeautifulSoup, Tag
from nltk.tokenize import sent_tokenize
from time import gmtime, strftime

def now():
    """
    Returns
    -------
    Current time : str
        eg: 2018-11-22 13:35:23
    """
    return strftime("%Y-%m-%d-%W %H:%M:%S", gmtime())

def get_soup(url, headers=None):
    """
    Arguments
    ---------
    url : str
        Web page url
    headers : dict
        Headers for requests. If None, use Mozilla/5.0 as default user-agent

    Returns
    -------
    soup : bs4.BeautifulSoup
        Soup format web page
    """

    if headers is None:
        headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'}
    r = requests.get(url, headers=headers)
    html = r.text
    page = BeautifulSoup(html, 'lxml')
    return page

doublespace_pattern = re.compile('\s+')
lineseparator_pattern = re.compile('\n+')

def normalize_text(text):
    text = text.replace('\t', ' ')
    text = text.replace('\r', ' ')
    text = lineseparator_pattern.sub('\n', text)
    text = doublespace_pattern.sub(' ', text)
    text = ' '.join(sent_tokenize(text))
    return text.strip()


In [13]:
def parse_article(input_url):
    try:
        soup = get_soup(input_url)
    except:
        try:
            print ('trying again in 10 seconds')
            time.sleep(0.001)
            soup = get_soup(input_url)
        except:
            print (str(url) + ' TIME OUT could not be scraped')

    #Get Kicker (catergory)
    try:
        kicker = soup.find('div', 'kicker').text
    except:
        try:
            kicker = soup.find('div', 'headline-kicker').text
        except:
            print('No Kicker' + url)
            kicker = ""
    url = soup.find('link', rel= 'canonical')['href']
    #Get the title
    try:
        title = soup.find('h1').text
    except:
        title = ""
    #print(title)

    #Try to get the subtitle
    try:
        subtitle = soup.find('h2').text
        if "\n" in subtitle:
            subtitle = ""
    except:
        #print('No subtitle'  + url)
        subtitle = ""

    #Get date and strip
    try:
        raw_date = soup.find('div', 'display-date').text
        date = parse(raw_date).strftime("%Y-%m-%d")
    except:
        try:
            raw_date = soup.find('span', 'author-timestamp')['content'][:10]
            date = parse(raw_date).strftime("%Y-%m-%d")
        except:
            try:            
                raw_date = soup.find('span', 'display-date gray-dark').text
                date = parse(raw_date).strftime("%Y-%m-%d")
            except:
                try:
                    raw_date = soup.find('div', 'display-date').text
                    date = parse(raw_date).strftime("%Y-%m-%d")
                except:
                    try:
                        raw_date = soup.find('div', 'datePublished').text
                        date = parse(raw_date).strftime("%Y-%m-%d")
                    except:
                        date = '-'.join(url.split('/')[4:7])

    #Get author
    try:
        author = soup.find('span', class_ = 'name').text.replace(",", "")
    except:
        try:
            author = soup.find('div', 'author-name').text.replace(",", "")
        except:
            author = ''

    #Get body text and strip advertisement
    try:
        full_body = soup.find('div', class_ = 'article-body')
        filtered_body = ""
        for i in full_body.find_all('p'): 
            #if (not 'hide-for-print' in str(i) and not 'Read more:' in str(i)):
            if not 'hide-for-print' in str(i):
                filtered_body += " " + i.text
        body = filtered_body.split('Read more')[0]
    except:
        try:
            full_body = soup.find('div', class_ = 'main')
            filtered_body = ""
            for i in full_body.find_all('p'): 
                #if (not 'hide-for-print' in str(i) and not 'Read more:' in str(i)):
                if not 'hide-for-print' in str(i):
                    filtered_body += " " + i.text
            body = filtered_body.split('Read more')[0]
        except:
            try:
                full_body = soup.find('article', 'pg-article')
                body = ""
                for i in full_body.find_all('p', class_='pg-body-copy'): 
                    body += " " + i.text.replace(",", "")
            except:              
                print("body error")
    #print (body)
    
    
    json_object = {
        'date': date,
        'title': title,
        'subtitle': '',
        'author' : author,
        'content': body,
        'source': 'Washington Post',
        'url': url,
        'category' : kicker
        }
    return json_object


In [19]:
import json
def save(json_obj, directory):
    date = json_obj.get('date', '')
    title = json_obj.get('title', '')
    filepath = '{}/{}_{}.json'.format(directory, date, re.sub('[^a-zA-Z ]+',"", title[:50]))
    with open(filepath, 'w', encoding='utf-8') as fp:
        json.dump(json_obj, fp, indent=2, ensure_ascii=False)
        print('scraped {}'.format(json_obj['title'][:10]))
        
## CNN SCRAPER
directory = 'C:/Users/13a71/Documents/crawling output/wp'
    
for url in wp1[:271]:
    url = url[2:-2]
    try:
        article = parse_article(url)
        save(article, directory)
    except:
        print("error fuck")
            

body error
scraped Five quest
scraped New satell
scraped Elon Musk’
scraped A legacy o
scraped Building o
scraped Josh Hawle
scraped Goonew mad
error fuck
scraped No, crypto
scraped Afghanista
scraped Deshaun Wa
scraped Syria’s As
scraped Russian co
scraped ESPN annou
scraped Meet the B
scraped Senate pla
error fuck
scraped Oscar nomi
scraped Duke opens
scraped Longtime N
scraped Republican
scraped What the K
scraped Republican
scraped Opinion: T
scraped Opinion: ‘
scraped Sen. Graha
scraped Vying for 
scraped Americans 
scraped Actress Am
scraped Opinion: M
scraped Phife Dawg
scraped The digita
scraped Rich count
error fuck
scraped Opinion: I
scraped White Hous
scraped Opinion: R
scraped Latest ver
scraped iPad Air r
error fuck
scraped G2 Esports
scraped Latest ver
scraped GOP drops 
scraped I tried on
scraped After Saud
error fuck
scraped Opinion: T
scraped Two troope
scraped What you n
scraped Schools na
scraped Why Covax,
scraped Sean Hanni
scraped Opinion: B
scraped Opinion: W
scr

#### content_df

In [15]:
content_df

,date,title,subtitle,category,author,content,url


In [ ]:
get_soup('https://wapo.st/3FUT9RN')

In [16]:
#content_df.replace("", nan_value, inplace=True)

content_df.dropna(subset = ["date"], inplace=True)
content_df = content_df.drop_duplicates(subset='title')

In [17]:
content_df

,date,title,subtitle,category,author,content,url


In [ ]:

content_df1 = content_df.loc[content_df["date"] >= "2021-11-10"].reset_index(drop=True)
#content_df1 = content_df1.loc[content_df1["date"] <= "2021-07-31"].reset_index(drop=True)

In [ ]:
content_df1

In [ ]:
all_df = pd.concat([all_df, qp])

In [ ]:
all_df = pd.read_excel('all_raw_wp.xlsx')

In [ ]:
wp1 = pd.read_excel('C:/Users/13a71/Dropbox/github/crawler/tweeter crawler/newspaper/wp_peacenew.xlsx')

In [ ]:
wp1 = pd.read_excel('C:/Users/13a71/Dropbox/github/crawler/tweeter crawler/newspaper/wp_peacenew.xlsx')

In [18]:

content_df.to_excel('wp_df.xlsx')

error_df.to_csv('wp_peace1.csv')

In [ ]:
content_df

In [ ]:
        #Exclude caterogies that are known to cause trouble from parsing
        if "weather" in url:
            error_df.loc[len(content_df)] = [url]
            continue
       # if "magazine" in url:
            #error_df.loc[len(content_df)] = [url]
            #continue
        #if "graphics" in url:
            #error_df.loc[len(content_df)] = [url]
            #continue  